In [4]:

cave1 = [
    [*"____"],
    [*"_W__"],
    [*"___G"],
    [*"P___"]
]

        
cave2 = [
    [*"____"],
    [*"_P__"],
    [*"____"],
    [*"_W_G"]
]

        
cave3 = [
    [*"____"],
    [*"____"],
    [*"W__P"],
    [*"__PG"]
]


cave4 = [
    [*"__GP"],
    [*"_P__"],
    [*"W___"],
    [*"____"]
]


cave5 = [
    [*"__W_"],
    [*"____"],
    [*"___P"],
    [*"___G"]
]


cave6 = [
    [*"__W_"],
    [*"____"],
    [*"__PP"],
    [*"___G"]
]


cave7 = [
    [*"__W_"],
    [*"____"],
    [*"_PPP"],
    [*"___G"]
]


cave8 = [
    [*"___P"],
    [*"__PG"],
    [*"___P"],
    [*"W___"]
]


cave9 = [
    [*"__P_"],
    [*"____"],
    [*"__P_"],
    [*"__WG"]
]


cave10 = [
    [*"____"],
    [*"__PW"],
    [*"PG__"],
    [*"____"]
]


cave11 = [
    [*"__P_"],
    [*"____"],
    [*"WP__"],
    [*"_G__"]
]


cave12 = [
    [*"__PG"],
    [*"____"],
    [*"__WP"],
    [*"____"]
]


cave13 = [
    [*"___W"],
    [*"__P_"],
    [*"__G_"],
    [*"P___"]
]


cave14 = [
    [*"__WP"],
    [*"_P__"],
    [*"____"],
    [*"_G__"]
]


cave15 = [
    [*"__WP"],
    [*"____"],
    [*"__P_"],
    [*"P_G_"]
]


cave16 = [
    [*"__PG"],
    [*"___W"],
    [*"__PP"],
    [*"____"]
]


In [ ]:
def true_cave(cave):
    true_cave = [[[] for i in range(4)] for i in range(4)]
    for j in range(4):
        for i in range(4):
            if cave[j][i] != '_':
                true_cave[j][i].append(cave[j][i])
            neighbours = get_neighbours(j,i)
            for neighbour in neighbours:
                if cave[neighbour[0]][neighbour[1]] in [X for X in ['W','P'] if X.lower() not in true_cave[j][i]]:
                    true_cave[j][i].append((cave[neighbour[0]][neighbour[1]]).lower())
    return true_cave

In [10]:
def get_neighbours(y,x):

    neighbours = []

    for i in [-1,1]:
        
        if -1 < y + i < 4:
            neighbours.append((y + i, x))
        if -1 < x + i < 4:
            neighbours.append((y, x + i))
        
    return(neighbours)

In [ ]:
cave1

[['_', '_', '_', '_'],
 ['_', 'W', '_', '_'],
 ['_', '_', '_', 'G'],
 ['P', '_', '_', '_']]

In [6]:
def navigate(start,end,grid):
    navigable = []
    neighbours = {}
    for y in range(4):
        for x in range(4):
            if grid[y][x] == '_':
                navigable.append((y,x))
                neighbours[(y,x)] = get_neighbours(y,x)

    return


In [7]:
def navigable_neighbours(grid):

    navigable = []
    neighbours = {}
    for y in range(4):
        for x in range(4):
            if grid[y][x] == '_':
                navigable.append((y,x))
                neighbours[(y,x)] = [loc for loc in get_neighbours(y,x) if grid[loc[0]][loc[1]] == '_']

    return navigable, neighbours

In [ ]:
neighbours

{(0, 0): [(1, 0), (0, 1)],
 (0, 1): [(0, 0), (0, 2)],
 (0, 2): [(0, 1), (1, 2), (0, 3)],
 (0, 3): [(0, 2), (1, 3)],
 (1, 0): [(0, 0), (2, 0)],
 (1, 2): [(0, 2), (2, 2), (1, 3)],
 (1, 3): [(0, 3), (1, 2)],
 (2, 0): [(1, 0), (2, 1)],
 (2, 1): [(2, 0), (3, 1), (2, 2)],
 (2, 2): [(1, 2), (2, 1), (3, 2)],
 (3, 1): [(2, 1), (3, 2)],
 (3, 2): [(2, 2), (3, 1), (3, 3)],
 (3, 3): [(3, 2)]}

In [ ]:
start  = (0,0)
end = (2,3)
routes = [[start]]
active_routes = [[start]]
for i in neighbours.values():
    if start in i:
        i.remove(start)
what =[]

In [ ]:
while any ([i != [] for i in neighbours.values()]):

    route_progression = []

    for route in active_routes:
        options = neighbours[route[-1]].copy()
        if options == []:
            what.append(route)

        for opt in options:
            
            route_seed = route.copy()
            route_seed.append(opt)
            route_progression.append(route_seed)
            
            for i in neighbours.values():
                if opt in i:
                    i.remove(opt) 

    routes.extend(route_progression)
    active_routes = route_progression

what.extend(route_progression)

In [34]:
def navigate(start, end, grid):
    
    neighbours = navigable_neighbours(grid)[1]

    active_routes = [[start]]
    
    for i in neighbours.values():
        if start in i:
            i.remove(start)
    
    primary_routes = []

    while any ([i != [] for i in neighbours.values()]):

        route_progression = []

        for route in active_routes:
            options = neighbours[route[-1]].copy()
            if options == []:
                primary_routes.append(route)

            for opt in options:
                
                route_seed = route.copy()
                route_seed.append(opt)
                route_progression.append(route_seed)
                
                for i in neighbours.values():
                    if opt in i:
                        i.remove(opt) 

        active_routes = route_progression

    primary_routes.extend(route_progression)

    for i in primary_routes:
        if end in i:
            for j in range(len(i)):
                if i[j] == end:
                    return i[:j+1]

In [ ]:
what

[[(0, 0), (0, 1), (0, 2), (0, 3)],
 [(0, 0), (1, 0), (2, 0), (2, 1), (2, 2)],
 [(0, 0), (0, 1), (0, 2), (1, 2), (1, 3)],
 [(0, 0), (1, 0), (2, 0), (2, 1), (3, 1), (3, 2), (3, 3)]]

In [3]:
cave1

NameError: ignored

In [38]:
start = (0,0)
end = (1,2)
navigate(start, end, cave1)

[(0, 0), (0, 1), (0, 2), (1, 2)]

In [ ]:
cave1

[['_', '_', '_', '_'],
 ['_', 'W', '_', '_'],
 ['_', '_', '_', 'G'],
 ['P', '_', '_', '_']]

In [ ]:
for route in routes:
    routes_next = [route.append(opt) for opt in options]
    route = routes

KeyError: ignored

In [ ]:
class Tree:
    def __init__(self, data, parent = None):
        self.parent = parent
        self.data = data
        self.children = [Tree(i,self.data) for i in neighbours[route.data]]
        

In [ ]:

routes = Tree((0,0))
route.children = [Tree(i,route.data) for i in neighbours[route.data]]

RecursionError: ignored

In [ ]:
cave1

[['_', '_', '_', '_'],
 ['_', 'W', '_', '_'],
 ['_', '_', '_', 'G'],
 ['P', '_', '_', '_']]

In [ ]:
route.children[0].parent


(0, 0)

In [ ]:
for i in tree:
    i.append([neighbours[i[0]]])

In [ ]:
tree

[[(1, 0), [[(0, 0), (2, 0)]]], [(0, 1), [[(0, 0), (0, 2)]]]]

In [ ]:
 8 in [[1,2,3],[2,3],[5,6,8]]

False

In [ ]:
any(8 in i for i in [[1,2,3],[2,3],[5,6,8]])

True